In [1]:
from typing import TypeVar, Generic, Callable
from dataclasses import dataclass
import asyncio
import os
from dotenv import load_dotenv
from azure.cosmos.aio import CosmosClient
from haskellian import either as E, Either, Left, Right
from kv import KV, CosmosPartitionKV, CosmosContainerKV, InvalidData, CosmosKV

load_dotenv()
COSMOS = os.environ['COSMOS_CONN_STR']

T = TypeVar('T')
U = TypeVar('U')

In [6]:
KV.of(f'azure+cosmos://{COSMOS}?db=test&container=0&partition=hello')

CosmosContainerKV(endpoint=https://coremeta.documents.azure.com:443/, database=test, container=0)